In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
from scipy import stats
from collections import Counter

# Morgan's I Index of spatial segregation

In [2]:
def SegIndex(pop, heigth, width):
    
    meanColor = pop.mean()
    
    
    numerator1 = pop.size  # Number of agents on the grid
    denominator1 = 0 
    numerator2 = 0
    denominator2 = 0
    
    for i in range(pop.size):
        yi = i//heigth
        xi = i - yi*width
        
        if(pop[yi, xi]!=0 ): 
            
            for j in range(pop.size):
                yj = j//heigth
                xj = j - yj*width
                
                if(pop[yj, xj]!=0 ):
                    if (yj in range(yi-1 if yi > 0 else yi, yi+2 if yi < len(pop)-1 else yi+1) and i!=j):
                        if (xj in range(xi-1 if xi > 0 else xi, xi+2 if xi < len(pop[0])-1 else xi+1)):
                            if (yj != xj):
                                denominator1 += 1
                                numerator2 += (pop[yi,xi] - meanColor)*(pop[yj,xj] - meanColor)
        denominator2 += (pop[yi,xi] - meanColor)*(pop[yi,xi] - meanColor)
            
            
                    
                    
    return (numerator1/denominator1)*(numerator2/denominator2)


# Definition of Environment and Agents

In [3]:
class GridObject:
    
    def __init__ (self, x, y, empty):
        self.x = x
        self.y = y
        self.empty = empty # True for empty cell, False fo agent

In [4]:
adap_period = 3

class Agent(GridObject):
    
    def __init__(self, x, y, characteristic, tolerance, adapt):
        
        super().__init__(x, y, False)
        
        self.type = characteristic
        self.threshold = tolerance
        self.tMinority = 0
        self.tMajority = 0
        self.adapt = adapt
        
    def Adapt(self):
        
        if self.tMinority >= self.adapt and self.threshold >1 and self.adapt!=0:
            self.threshold -=1
        elif self.tMajority >= self.adapt and self.threshold >8 and self.adapt!=0:
            self.threshold +=1

In [5]:
nTypeComunities=2
probabilitiesType= [0.1,0.45,0.45]

class Grid():
    
    def __init__(self, width, heigth, vMinimum, vMaximum, fractionAdaptive):
        
        self.width = width
        self.heigth = heigth
        self.population = []
        
        for y in range(self.width):
            for x in range(self.heigth):

                random_number = np.random.choice(nTypeComunities+1, p=probabilitiesType)

                if random_number==0:
                    self.population.append(GridObject(x,y, True))
                else:
                    
                    tolerance = np.random.randint(vMinimum, vMaximum)
                    adapt = np.random.choice([0,3], p=[1-fractionAdaptive, fractionAdaptive])
                    
                    if random_number==1:
                        self.population.append(Agent(x, y, 1, tolerance, adapt))
                    elif random_number==2:
                        self.population.append(Agent(x, y, 2, tolerance, adapt))
    
    def getPopMatrix(self):
        popMatrix = np.zeros((self.heigth, self.width))
        
        for obj in self.population:
            if not obj.empty:
                popMatrix[obj.y, obj.x] = obj.type
        return popMatrix
    
    def getThreMatrix(self):
        threMatrix = np.zeros((self.heigth, self.width))
        
        for obj in self.population:
            if not obj.empty:
                threMatrix[obj.y, obj.x] = obj.threshold
        return threMatrix
    
    def countNeighbours(self, x, y):
        SameNeighbours=0
        neighbours=[]

        if (not self.population[x + y*self.width].empty):
            # If it is an agent, return neigh.s

            # The loop goes trhough all the neighbours and the agent itself
            for r in range(y-1 if y > 0 else y, y+2 if y < self.heigth -1 else y+1):
                for c in range(x-1 if x > 0 else x, x+2 if x < self.width -1 else x+1):
                    neighbours.append(self.population[r*self.width + c])

            # nr total of neighbours: minus the agent itself minus the empty spots
            TotalNeighbours = len([neighbor for neighbor in neighbours if not neighbor.empty ])- 1

            # nr SAMETYPE neighbours: minus the agent itself 
            SameNeighbours =  len([neighbor for neighbor in neighbours if not neighbor.empty and neighbor.type == self.population[x + y*self.width].type ])- 1

            # nr OPOSITETYPE neighbours: total neigh. minus the sametype neigh.
            OpositeNeighbours = TotalNeighbours - SameNeighbours

            return TotalNeighbours , SameNeighbours, OpositeNeighbours    

        else:
            # If it is an empty spot, return 000
            return 0 , 0, 0
        
    def replacePosition(self, agent):
        
        emptyCells = [emptyCell for emptyCell in self.population if emptyCell.empty]
        
        random_emptyCell = np.random.randint(len(emptyCells))
        
        posxEmptyCell = emptyCells[random_emptyCell].x
        posyEmptyCell = emptyCells[random_emptyCell].y
        
        emptyCells[random_emptyCell].x = agent.x
        emptyCells[random_emptyCell].y = agent.y
        
        self.population[agent.x + agent.y*self.width] = emptyCells[random_emptyCell]
        
        agent.x = posxEmptyCell
        agent.y = posyEmptyCell
        
        self.population[posxEmptyCell + posyEmptyCell*self.width] = agent
        
        
    
    def updateTimesMinoMajo(self):
        
        agents = [agent for agent in self.population if not agent.empty ]
        
        for agent in agents:
            TotalNeighbours , SameNeighbours, OpositeNeighbours = self.countNeighbours( agent.x, agent.y)
            
            agent.Adapt()
            
            if(TotalNeighbours != 0):
                percentagem = SameNeighbours/TotalNeighbours
                if(percentagem >= 0.5):
                    agent.tMinority = 0
                    agent.tMajority += 1
                elif(percentagem < 0.5):
                    agent.tMinority += 1
                    agent.tMajority = 0
                    
    def PercetageunHappy(self):
        nUnhappy=0
        pop = self.getPopMatrix()
        MinimunEqualNeighbors = self.getThreMatrix()
        for i in range(self.heigth):
            for j in range(self.width):
                if (pop[i,j]!=0 and (countNeighbours(population, i, j)[2] < MinimunEqualNeighbors[i,j])):
                    nUnhappy+=1
        return nUnhappy
            

# Visualization Functions

In [6]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.colors import ListedColormap

def PlottarThreshold(threMatrix, filename, title ):
    fig, ax = plt.subplots()
    colors = ListedColormap(["red", "orangered","lightcoral","lightsalmon", "lightskyblue","dodgerblue","cornflowerblue", "tab:blue", "darkblue"])
    
    ax.matshow(threMatrix, cmap=plt.cm.Blues)
    fig.colorbar(ax.matshow(threMatrix, cmap=plt.cm.RdBu, vmax=8, vmin=0))
    plt.title(title)
    plt.show()
    
    fig.savefig(filename, bbox_inches='tight', dpi=150)

In [7]:
from matplotlib.colors import ListedColormap

def PlottarPopulation(pop, filename, title ):
    fig, ax = plt.subplots()
    colors = ListedColormap(["white", "yellowgreen", "tab:blue"])
    ax.matshow(pop, cmap=colors, vmax=2, vmin=0)
    plt.title(title)
    plt.show
    fig.savefig(filename, bbox_inches='tight', dpi=150)

In [8]:
def ThresholdDist(grid, filename, title):
    vec = np.zeros(9)
    for obj in grid.population:
        if not obj.empty:
            vec[int(obj.threshold)]+=1
    
    return vec

# Simulation

In [9]:
def Simulation(vMinimum, vMaximum, names, fraction):
    
    grid = Grid(50,50, vMinimum, vMaximum, fraction)
    
#     PlottarPopulation(grid.getPopMatrix(), names[0], 'Population')
#     PlottarThreshold(grid.getThreMatrix(), names[1], 'Threshold')
    
    seg_initial = SegIndex(grid.getPopMatrix(),50,50)
    seg_threshold_initial = SegIndex(grid.getThreMatrix(),50,50)
    
    print ("Initial index of segmentation: ", seg_initial)
    print ("Initial index of segmentation (threshold): ", seg_threshold_initial)
    
    vec=[]
    
    counter=0
    while(grid.PercetageunHappy()<0.95):
        
        counter+=1 
        
        agents = [agent for agent in grid.population if not agent.empty]
        
        for agent in agents:
            
            TotalNeighbours , SameNeighbours, OpositeNeighbours = grid.countNeighbours(agent.x, agent.y)
            
            if agent.threshold > SameNeighbours or np.random.rand()<0.01:
                grid.replacePosition(agent)
        
        grid.updateTimesMinoMajo()
        
        if(counter%100==0):
            vec.append(ThresholdDist(grid,'pape.pdf', 'meio Threshold'))
    
    PlottarPopulation(grid.getPopMatrix(), names[0], 'Population')
    PlottarThreshold(grid.getThreMatrix(), names[1], 'Threshold')
    
    seg_final = SegIndex(grid.getPopMatrix(),50,50)
    seg_threshold_final = SegIndex(grid.getThreMatrix(),50,50)
    
    print ("Final index of segmentation: ", seg_final)
    print ("Final index of segmentation (threshold): ", seg_threshold_final)
    
    return grid.getPopMatrix(), grid.getThreMatrix(), seg_initial, seg_final, seg_threshold_initial, seg_threshold_final, vec

In [10]:
def Run (run_number, vMinimum, vMaximum):
    
    plot_names = ["pape.pdf","pape.pdf"]
    
    p_Adaptation = np.arange(0,1.05,0.1)
    
    df_results = pd.DataFrame(columns=['run_number',
                                       'p_adap', 
                                       'seg_pop_initial',
                                       'seg_pop_final', 
                                       'seg_threshold_initial',
                                       'seg_threshold_final',
                                       'mean_final_threshold',
                                       'mode_final_threshold',
                                       '2nd_mode_final_threshold'])

    
    for p_adap in p_Adaptation:
        print("--------------------- P_ADAP == ",p_adap )
        results = Simulation(vMinimum, vMaximum, plot_names, p_adap)
        
        # -------- MODE CALCULATION
        vals,counts = np.unique(results[1], return_counts=True)
        index = np.argmax(counts)
        mode_final_threshold = vals[index]
        
        # -------- 2ND MODE CALCULATION
        ctr = Counter(results[1].ravel())
        second_most_common_value, its_frequency = ctr.most_common(2)[1]
        
        df_results = df_results.append( {
            'run_number':                 run_number,
            'p_adap':                     p_adap,
            'seg_pop_initial':            results[2],
            'seg_pop_final':              results[3], 
            'seg_threshold_initial':      results[4],
            'seg_threshold_final':        results[5],
            'mean_final_threshold':       np.mean(results[1]),
            'mode_final_threshold':       mode_final_threshold,
            '2nd_mode_final_threshold':   second_most_common_value
        },ignore_index=True)
        
    return df_results

# Threshold : $n = 4$

In [ ]:
df_overall = pd.DataFrame(columns=['run_number',
                                   'p_adap', 
                                   'seg_pop_initial',
                                   'seg_pop_final', 
                                   'seg_threshold_initial',
                                   'seg_threshold_final',
                                   'mean_final_threshold',
                                   'mode_final_threshold',
                                   '2nd_mode_final_threshold'])

for run in range(30):
    dfResults = Run(run, 4, 5)
    df_overall = df_overall.append(dfResults)
    
df_overall.to_csv(r'df_overall.csv', index = False)

# Threshold : $n \in [4,8]$

In [ ]:
df_overall_BroadenThres = pd.DataFrame(columns=['run_number',
                                       'p_adap', 
                                       'seg_pop_initial',
                                       'seg_pop_final', 
                                       'seg_threshold_initial',
                                       'seg_threshold_final',
                                       'mean_final_threshold',
                                       'mode_final_threshold',
                                       '2nd_mode_final_threshold'])

for run in range(30):
    dfResults = Run(run, 4, 8)
    df_overall_BroadenThres = df_overall_BroadenThres.append(dfResults)

df_overall_BroadenThres.to_csv(r'df_overall_threshold.csv', index = False)

# Changing the percentage of empty places: empty = 1% instead of 10%

In [ ]:
probabilitiesType= [0.01,0.495,0.495]

# ----------------------------
#  COMPARE WITH n=4
# ----------------------------
df_overall_Empty = pd.DataFrame(columns=['run_number',
                                   'p_adap', 
                                   'seg_pop_initial',
                                   'seg_pop_final', 
                                   'seg_threshold_initial',
                                   'seg_threshold_final',
                                   'mean_final_threshold',
                                   'mode_final_threshold',
                                   '2nd_mode_final_threshold'])

for run in range(30):
    dfResults = Run(run, 4, 5)
    df_overall_Empty = df_overall_Empty.append(dfResults)
    
df_overall_Empty.to_csv(r'df_overall_Empty.csv', index = False)